### 6 min lega but multi base link se bina block hue link nikla dega  (2616 links )

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def extract_links_from_page(url):
    """1 to 100 page ka list bana kar bina block hue data fetechd kar raha h.
    """

    try:
        response = requests.get(url, headers={'User-Agent': 'your user agent'})  # Set user agent
        response.raise_for_status()  # Raise exception for non-200 status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', class_='table table-striped')
        if table:
            links = []
            for link in table.find_all('a', href=True):
                links.append(link['href'])
            return links
        else:
            return []
    except requests.exceptions.RequestException as e:
        print(f"Error occurred for URL {url}: {e}")
        return []

def main():
    base_urls = [
        "https://chartink.com/screeners/fundamental-screeners?page=",
        "https://chartink.com/screeners/candlestick-patterns-screeners?page=",
        # ... add more base URLs
    ]
    num_pages_per_base_url = 100
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        # Add more user agents here
    ]
    user_agent_index = 0

    # Consider robots.txt and terms of service first

    all_links = []
    delay = 1  # Initial delay
    for base_url_index, base_url in enumerate(base_urls):
        for page_number in range(1, num_pages_per_base_url + 1):
            url = base_url + str(page_number)

            response = requests.get(url, headers={'User-Agent': user_agents[user_agent_index]})
            if response.status_code == 200:
                links = extract_links_from_page(url)
                if links:
                    all_links.extend(links)
                    for link in links:
                        print(f"Extracted link: {link}")
                else:
                    print(f"No links found on page {page_number} of base URL {base_url_index + 1}")
            elif response.status_code == 429:
                print(f"429 error on page {page_number} of base URL {base_url_index + 1}. Waiting for {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                print(f"Unexpected status code {response.status_code} for URL {url}")

        user_agent_index = (user_agent_index + 1) % len(user_agents)  # Rotate user agents

    # Create DataFrame and save to Excel
    df = pd.DataFrame({'Link': all_links})
    df.to_excel('extracted_links.xlsx', index=False)  # Save as XLSX

if __name__ == "__main__":
    main()
